<a href="https://colab.research.google.com/github/RANJITHMUTHUSAMY005/NUTRITION-PARADOX/blob/main/NPQUERIES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df_obesity=pd.read_csv("obesity.csv")

In [7]:
df_mal=pd.read_csv("malnutrition.csv")

In [3]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 42.7 MB/s eta 0:00:00


In [4]:
import sqlite3
conn=sqlite3.connect("nutritionparadox.db")
cursor=conn.cursor()
obesity="""create table IF NOT EXISTS obesity(
Region	TEXT,
Gender	TEXT,
Year	INTEGER,
LowerBound	REAL,
UpperBound	REAL,
Mean_Estimate	REAL,
age_group	TEXT,
Country	TEXT,
CI_Width	REAL,
obesity_level	TEXT);
"""
cursor.execute(obesity)
conn.commit()

In [5]:
import sqlite3
conn=sqlite3.connect("nutritionparadox.db")
cursor=conn.cursor()
malnutrition="""create table IF NOT EXISTS malnutrition(
Region	TEXT,
Gender	TEXT,
Year	INTEGER,
LowerBound	REAL,
UpperBound	REAL,
Mean_Estimate	REAL,
age_group	TEXT,
Country	TEXT,
CI_Width	REAL,
Malnutrition_level	TEXT);
"""
cursor.execute(malnutrition)
conn.commit()

In [6]:
#inserting values into table
for index, row in df_obesity.iterrows():
    cursor.execute("""
        INSERT INTO obesity(
            Region, Gender, Year, Mean_Estimate, LowerBound, UpperBound,
            age_group, Country, CI_Width, obesity_level
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Region'], row['Gender'], row['Year'], row['Mean_Estimate'],
        row['LowerBound'], row['UpperBound'], row['age_group'],
        row['Country'], row['CI_Width'], row['obesity_level']
    ))
conn.commit()

In [8]:
#inserting values into malnutrition table
for index, row in df_mal.iterrows():
    cursor.execute("""
        INSERT INTO malnutrition(
            Region, Gender, Year, Mean_Estimate, LowerBound, UpperBound,
            age_group, Country, CI_Width, Malnutrition_level
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Region'], row['Gender'], row['Year'], row['Mean_Estimate'],
        row['LowerBound'], row['UpperBound'], row['age_group'],
        row['Country'],
        row['CI_Width'], row['Malnutrition_level']
    ))
conn.commit()
print("Data inserted into malnutrition table successfully.")

Data inserted into malnutrition table successfully.


In [ ]:




Country count by obesity level category and age group
Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)
Average obesity by age group
Top 10 Countries with consistent low obesity (low average + low CI)over the years
Countries where female obesity exceeds male by large margin (same       year)
Global average obesity percentage per year


# 1.Top 5 regions with the highest average obesity levels in the most recent year(2022)

In [10]:
q1 = """
SELECT Region, AVG(CASE
                      WHEN obesity_level = 'High' THEN 1
                      WHEN obesity_level = 'Moderate' THEN 0.5
                      ELSE 0  -- Assuming 'Low' or other levels are given a value of 0
                    END) AS avg_obesity_level
FROM obesity
WHERE Year = 2022
GROUP BY Region
ORDER BY avg_obesity_level DESC
LIMIT 5;
"""
cursor.execute(q1)
results=cursor.fetchall()

In [11]:
results_df=pd.read_sql_query(q1,conn)
results_df

,Region,avg_obesity_level
0,Western Pacific,0.584677
1,Eastern Mediterranean,0.541667
2,Americas,0.527027
3,Africa,0.504433
4,South-East Asia,0.500000


# 2.Top 5 countries with highest obesity estimates

In [12]:
q2 = """
SELECT Country, Mean_Estimate
FROM obesity
ORDER BY Mean_Estimate DESC
LIMIT 5;
"""
cursor.execute(q2)
results=cursor.fetchall()

In [13]:
results_df=pd.read_sql_query(q2,conn)
results_df

,Country,Mean_Estimate
0,American Samoa,80.609250
1,American Samoa,80.316889
2,American Samoa,79.969907
3,American Samoa,79.600513
4,American Samoa,79.205018


# 3.Obesity trend in India over the years(Mean_estimate)

In [14]:
q3='''
select Mean_Estimate, Country, Year
from obesity
where Country='India'
group by Year, Mean_Estimate
order by Year desc
'''
cursor.execute(q3)
results=cursor.fetchall()

In [15]:
results_df=pd.read_sql_query(q3,conn)
results_df

,Mean_Estimate,Country,Year
0,2.374715,India,2022
1,2.718108,India,2022
2,3.029781,India,2022
3,3.037025,India,2022
4,3.438066,India,2022
...,...,...,...
127,2.714740,India,2012
128,2.730843,India,2012
129,3.178412,India,2012
130,3.969172,India,2012


# 4.Average obesity by gender

In [16]:
q4='''
select Gender, AVG(Mean_Estimate) as avg_mean_estimate
from obesity
group by Gender
'''
cursor.execute(q4)
results=cursor.fetchall()

In [17]:
results_df=pd.read_sql_query(q4,conn)
results_df

,Gender,avg_mean_estimate
0,Both,12.781969
1,Female,12.724456
2,Male,12.776696


# 5.Country count by obesity level category and age group

In [18]:
q5='''
select obesity_level, count(distinct(Country)) as Country_count, age_group
from obesity
group by obesity_level,age_group
order by obesity_level, age_group
'''
cursor.execute(q5)
results=cursor.fetchall()

In [19]:
results_df=pd.read_sql_query(q5,conn)
results_df

,obesity_level,Country_count,age_group
0,High,94,adult
1,High,11,child
2,Low,99,adult
3,Low,26,child
4,Moderate,176,adult
5,Moderate,210,child


# 6.Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)

In [20]:
q6 = """
SELECT * FROM (
    -- Least Reliable Countries (Top 5 with highest CI_Width)
    SELECT Country, MAX(CI_Width) AS ci_width, 'Least Reliable' AS category
    FROM obesity
    GROUP BY Country
    ORDER BY ci_width DESC
    LIMIT 5
)

UNION ALL

SELECT * FROM (
    -- Most Consistent Countries (Top 5 with smallest average CI_Width)
    SELECT Country, AVG(CI_Width) AS ci_width, 'Most Consistent' AS category
    FROM obesity
    GROUP BY Country
    ORDER BY ci_width ASC
    LIMIT 5
);
"""
cursor.execute(q6)
results = cursor.fetchall()

In [21]:
results_df=pd.read_sql_query(q6,conn)
results_df

,Country,ci_width,category
0,Haiti,40.554062,Least Reliable
1,Bermuda,40.079678,Least Reliable
2,Puerto Rico,40.032199,Least Reliable
3,American Samoa,39.954137,Least Reliable
4,Grenada,39.696849,Least Reliable
5,Global,0.708953,Most Consistent
6,Low & Middle Income,0.861060,Most Consistent
7,South-East Asia Region,1.017775,Most Consistent
8,India,1.140133,Most Consistent
9,Japan,1.155903,Most Consistent


# 7.Average obesity by age group

In [22]:
q7='''
select age_group, avg(mean_estimate) as avg_mean_estimate
from obesity
group by age_group
'''
cursor.execute(q7)
results=cursor.fetchall()

In [23]:
results_df=pd.read_sql_query(q7,conn)
results_df

,age_group,avg_mean_estimate
0,adult,21.833223
1,child,9.736979


# 8.Top 10 Countries with consistent low obesity (low average + low CI)over the years

In [24]:
q8 = """
SELECT Country,
       AVG(Mean_Estimate) AS avg_obesity,
       AVG(CI_Width) AS avg_ci_width,
       COUNT(DISTINCT Year) AS years_count
FROM obesity
GROUP BY Country
ORDER BY avg_obesity ASC, avg_ci_width ASC
LIMIT 10;
"""
cursor.execute(q8)
results=cursor.fetchall()

In [25]:
results_df=pd.read_sql_query(q8,conn)
results_df

,Country,avg_obesity,avg_ci_width,years_count
0,Timor-Leste,0.994067,1.273569,11
1,Ethiopia,1.491994,1.589414,11
2,Burkina Faso,1.721496,1.531514,11
3,Rwanda,1.767194,2.360579,11
4,Bangladesh,1.976673,1.663506,11
5,Madagascar,1.984110,4.356885,11
6,Burundi,2.304665,5.198043,11
7,Malawi,2.391324,2.632714,11
8,Nepal,2.494255,3.098051,11
9,Eritrea,2.770560,5.870395,11


# 9.Countries where female obesity exceeds male by large margin

In [26]:
q9 = """
SELECT
    female.Country,
    female.Year,
    female.Mean_Estimate AS female_obesity_rate,
    male.Mean_Estimate AS male_obesity_rate
FROM
    obesity AS female,
    obesity AS male
WHERE
    female.Country = male.Country
    AND female.Year = male.Year
    AND female.Gender = 'Female'
    AND male.Gender = 'Male'
    AND (female.Mean_Estimate - male.Mean_Estimate) >= 5
ORDER BY
    (female.Mean_Estimate - male.Mean_Estimate) DESC;
"""
cursor.execute(q9)
results=cursor.fetchall()

In [27]:
results_df=pd.read_sql_query(q9,conn)
results_df

,Country,Year,female_obesity_rate,male_obesity_rate
0,American Samoa,2012,76.983809,23.729610
1,American Samoa,2012,76.983809,23.866493
2,American Samoa,2012,76.983809,24.116910
3,American Samoa,2013,77.297693,24.599233
4,American Samoa,2013,77.297693,24.727380
...,...,...,...,...
7373,Indonesia,2021,15.544349,10.537361
7374,Brazil,2017,27.254888,22.249368
7375,Denmark,2018,11.921830,6.917784
7376,Bosnia and Herzegovina,2021,25.896215,20.892647


# 10.Global average obesity percentage per year

In [28]:
q10 = """
SELECT
    Year,
    AVG(Mean_Estimate) AS global_avg_obesity_rate
FROM obesity
GROUP BY Year
ORDER BY Year;
"""
cursor.execute(q10)
results=cursor.fetchall()

In [29]:
results_df=pd.read_sql_query(q10,conn)
results_df

,Year,global_avg_obesity_rate
0,2012,11.061969
1,2013,11.359637
2,2014,11.670343
3,2015,11.994231
4,2016,12.333907
5,2017,12.689997
6,2018,13.060296
7,2019,13.444604
8,2020,13.843002
9,2021,14.250923


# **MALNUTRITION QUERIES**

# 11.Avg. malnutrition by age group

In [30]:
q11='''
select age_group, avg(mean_estimate) as avg_mean_estimate
from malnutrition
group by age_group
order by avg_mean_estimate desc
'''
cursor.execute(q11)
results=cursor.fetchall()

In [31]:
results_df=pd.read_sql_query(q11,conn)
results_df

,age_group,avg_mean_estimate
0,adult,5.940619
1,child,5.159062


# 12.Avg. malnutrition by age group

In [32]:
q12='''
select Country, AVG(Mean_Estimate) as avg_mean_estimate
from malnutrition
group by Country
order by avg_mean_estimate desc
limit 5
'''
cursor.execute(q12)
results=cursor.fetchall()

In [33]:
results_df=pd.read_sql_query(q12,conn)
results_df

,Country,avg_mean_estimate
0,India,22.142361
1,South-East Asia Region,19.238903
2,Bangladesh,18.245768
3,Sri Lanka,17.871907
4,Eritrea,17.472508


# 13.Malnutrition trend in African region over the years

In [34]:
q13='''
select Year, AVG(Mean_Estimate) AS average_malnutrition_rate
FROM malnutrition
WHERE Region = 'Africa'
GROUP BY Year
ORDER BY Year;
'''
cursor.execute(q13)
results=cursor.fetchall()

In [35]:
results_df=pd.read_sql_query(q13,conn)
results_df

,Year,average_malnutrition_rate
0,2012,8.849655
1,2013,8.749857
2,2014,8.662762
3,2015,8.587719
4,2016,8.523125
5,2017,8.466465
6,2018,8.414722
7,2019,8.369700
8,2020,8.331320
9,2021,8.298626


# 14.Gender-based average malnutrition

In [36]:
q14 = """
SELECT
    Gender,
    AVG(Mean_Estimate) AS average_malnutrition_rate
FROM malnutrition
GROUP BY Gender;
"""
cursor.execute(q14)
results=cursor.fetchall()

In [37]:
results_df=pd.read_sql_query(q14,conn)
results_df

,Gender,average_malnutrition_rate
0,Both,5.361553
1,Female,4.675483
2,Male,6.026317


# 15.Malnutrition level-wise (average CI_Width by age group)

In [38]:
q15='''
select age_group, avg(CI_Width) as avg_ci_width
from malnutrition
group by age_group
order by avg_ci_width desc
'''
cursor.execute(q15)
results=cursor.fetchall()

In [39]:
results_df=pd.read_sql_query(q15,conn)
results_df

,age_group,avg_ci_width
0,child,5.103133
1,adult,3.693780


# 16.Yearly malnutrition change in specific countries(India, Nigeria, Brazil)

In [40]:
q16 = """
SELECT
    Country,
    Year,
    Mean_Estimate,
    LAG(Mean_Estimate) OVER (PARTITION BY Country ORDER BY Year) AS previous_year_malnutrition_rate,
    Mean_Estimate - LAG(Mean_Estimate) OVER (PARTITION BY Country ORDER BY Year) AS malnutrition_change
FROM malnutrition
WHERE  Country IN ('India', 'Nigeria', 'Brazil')  -- Filter for specific countries
ORDER BY Country, Year;
"""
cursor.execute(q16)
results=cursor.fetchall()

In [41]:
results_df=pd.read_sql_query(q16,conn)
results_df

,Country,Year,Mean_Estimate,previous_year_malnutrition_rate,malnutrition_change
0,Brazil,2012,2.582618,NaN,NaN
1,Brazil,2012,4.151142,2.582618,1.568524
2,Brazil,2012,3.388768,4.151142,-0.762374
3,Brazil,2012,2.545835,3.388768,-0.842933
4,Brazil,2012,2.497542,2.545835,-0.048293
...,...,...,...,...,...
391,Nigeria,2022,9.408734,14.139627,-4.730893
392,Nigeria,2022,12.827678,9.408734,3.418944
393,Nigeria,2022,18.538181,12.827678,5.710503
394,Nigeria,2022,9.595276,18.538181,-8.942905


# 17.Yearly malnutrition change in specific countries(India, Nigeria, Brazil)

In [42]:
q17='''
select Region, AVG(Mean_Estimate) as avg_mean_estimate
from malnutrition
group by Region
order by avg_mean_estimate;
'''
cursor.execute(q17)
results=cursor.fetchall()

In [43]:
results_df=pd.read_sql_query(q17,conn)
results_df

,Region,avg_mean_estimate
0,Europe,2.426759
1,Americas,3.174752
2,Western Pacific,3.434955
3,Eastern Mediterranean,6.661349
4,Special case,8.354200
5,Africa,8.502186
6,South-East Asia,13.624525


# 18.Countries with increasing malnutrition

In [44]:
q18 = """
SELECT
    Country,
    MIN(Year) AS early_year,
    MAX(Year) AS recent_year,
    MIN(Mean_Estimate) AS early_malnutrition_rate,
    MAX(Mean_Estimate) AS recent_malnutrition_rate,
    MAX(Mean_Estimate) - MIN(Mean_Estimate) AS malnutrition_change
FROM malnutrition
GROUP BY Country
HAVING MAX(Mean_Estimate) > MIN(Mean_Estimate)  -- Filtering where malnutrition has increased
ORDER BY malnutrition_change DESC;
"""
cursor.execute(q18)
results=cursor.fetchall()

In [45]:
results_df=pd.read_sql_query(q18,conn)
results_df

,Country,early_year,recent_year,early_malnutrition_rate,recent_malnutrition_rate,malnutrition_change
0,Eritrea,2012,2022,7.868055,35.894114,28.026059
1,Ethiopia,2012,2022,7.553969,29.304048,21.750079
2,Somalia,2012,2022,4.688896,25.521731,20.832835
3,India,2012,2022,13.073501,31.900835,18.827334
4,Pakistan,2012,2022,8.213687,25.950593,17.736906
...,...,...,...,...,...,...
205,Cook Islands,2012,2022,0.449898,1.637155,1.187258
206,Peru,2012,2022,0.825172,1.985275,1.160103
207,American Samoa,2012,2022,0.246522,1.146955,0.900433
208,Niue,2012,2022,0.353357,1.249841,0.896485


# 19.Min/Max malnutrition levels year-wise comparison

In [46]:
q19 = """
SELECT
    Year,
    MIN(Mean_Estimate) AS min_malnutrition_rate,
    MAX(Mean_Estimate) AS max_malnutrition_rate
FROM malnutrition
GROUP BY Year;
"""
cursor.execute(q19)
results=cursor.fetchall()

In [47]:
results_df=pd.read_sql_query(q19,conn)
results_df

,Year,min_malnutrition_rate,max_malnutrition_rate
0,2012,0.175309,35.894114
1,2013,0.176421,35.694364
2,2014,0.177772,35.498199
3,2015,0.178445,35.312954
4,2016,0.178804,35.147691
5,2017,0.179286,34.997904
6,2018,0.179796,34.856236
7,2019,0.180428,34.715995
8,2020,0.181989,34.578446
9,2021,0.184717,34.429225


# 20.High CI_Width flags for monitoring(CI_width > 5)

In [48]:
q20 = """
SELECT
    Region,
    Country,
    Year,
    Mean_Estimate,
    CI_Width
FROM malnutrition
WHERE CI_Width > 5
ORDER BY CI_Width DESC;
"""
cursor.execute(q20)
results=cursor.fetchall()

In [49]:
results_df=pd.read_sql_query(q20,conn)
results_df

,Region,Country,Year,Mean_Estimate,CI_Width
0,Africa,Senegal,2022,16.941771,26.701744
1,Africa,Senegal,2021,16.992038,25.679344
2,Africa,Senegal,2020,17.046429,24.586366
3,Africa,Mauritania,2022,16.080868,23.810544
4,Africa,Senegal,2019,17.105949,23.466898
...,...,...,...,...,...
9567,Eastern Mediterranean,Jordan,2019,3.876585,5.001890
9568,Africa,Comoros,2018,3.272964,5.001566
9569,Western Pacific,Philippines,2021,7.551254,5.001525
9570,Africa,Sierra Leone,2021,4.989996,5.000505


# Combined

In [50]:
q21 = """
SELECT
    o.Country,
    AVG(o.Mean_Estimate) AS Average_Obesity,
    AVG(m.Mean_Estimate) AS Average_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country  -- Join the tables on the Country field
WHERE o.Country IN ('India', 'Nigeria', 'Brazil', 'Europe', 'Mexico')  -- Filtering for specific countries
GROUP BY o.Country;
"""
cursor.execute(q21)
results=cursor.fetchall()

In [51]:
results_df=pd.read_sql_query(q21,conn)
results_df

,Country,Average_Obesity,Average_Malnutrition
0,Brazil,16.613028,2.685355
1,Europe,11.853727,2.545575
2,India,3.429781,22.142361
3,Mexico,20.062568,1.845168
4,Nigeria,4.467604,11.889187


In [52]:
q23 = """
SELECT
    o.Region,
    AVG(o.Mean_Estimate) AS Avg_Obesity,
    AVG(m.Mean_Estimate) AS Avg_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country
WHERE o.Region IN ('Africa', 'Americas')  -- Filter for the specified regions
GROUP BY o.Region;
"""
cursor.execute(q23)
results=cursor.fetchall()

In [53]:
results_df=pd.read_sql_query(q23,conn)
results_df

,Region,Avg_Obesity,Avg_Malnutrition
0,Africa,5.415204,8.502186
1,Americas,18.402103,3.174752


In [54]:
q24 = """
SELECT
    o.Country,
    MIN(o.Mean_Estimate) AS Min_Obesity,
    MAX(o.Mean_Estimate) AS Max_Obesity,
    MIN(m.Mean_Estimate) AS Min_Malnutrition,
    MAX(m.Mean_Estimate) AS Max_Malnutrition
FROM obesity o
JOIN  malnutrition m ON o.Country = m.Country
GROUP BY o.Country
HAVING
    MAX(o.Mean_Estimate) > MIN(o.Mean_Estimate)  -- Obesity has increased
    AND MAX(m.Mean_Estimate) < MIN(m.Mean_Estimate);  -- Malnutrition has decreased
"""
cursor.execute(q24)
results=cursor.fetchall()

In [55]:
results_df=pd.read_sql_query(q24,conn)
results_df

,Country,Min_Obesity,Max_Obesity,Min_Malnutrition,Max_Malnutrition


In [56]:
q25 = """
SELECT
    o.age_group,
    o.Year,
    AVG(o.Mean_Estimate) AS Avg_Obesity,
    AVG(m.Mean_Estimate) AS Avg_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country
GROUP BY o.age_group, o.Year;
"""
cursor.execute(q25)
results=cursor.fetchall()

In [57]:
results_df=pd.read_sql_query(q25,conn)
results_df

,age_group,Year,Avg_Obesity,Avg_Malnutrition
0,adult,2012,19.517937,5.354451
1,adult,2013,19.949264,5.354451
2,adult,2014,20.391706,5.354451
3,adult,2015,20.847085,5.354451
4,adult,2016,21.315222,5.354451
5,adult,2017,21.790270,5.354451
6,adult,2018,22.269403,5.354451
7,adult,2019,22.757322,5.354451
8,adult,2020,23.262192,5.354451
9,adult,2021,23.777187,5.354451
